In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import matplotlib.pyplot as plt
import repos_tools
import os.path
import dream
import attila_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
GenRev not found, steiner wont work
Import error: Negex. Using keyword matching instead
Import error: Funcassociate. Make sure that funcassociate is in toolbox!


In [2]:
hgn_fpath = '/Users/jonesa7/CTNS/resources/hgnc/hgnc_complete_set.txt'
ingmt_fpath = '../../results/2022-08-22-dream-modules/dream_consensus_modules-PPI-STRING_Consensus.gmt'
outgmt_fpath = '../../results/2022-08-22-dream-modules/dream_consensus_modules-PPI-STRING_Consensus-entrez_id.gmt'
if not os.path.exists(outgmt_fpath):
    repos_tools.symbol2entrez_gmt(ingmt_fpath, outgmt_fpath, hgn_fpath)

In [3]:
enrichment_fpath = '../../results/2022-08-22-dream-modules/output/35379992-GCST90027158-MONDO_0004975-Build38-corrected.f.tsv.PathwaySet--dream_consensus_modules-PPI-STRING_Consensus-entrez_id--sum.txt'
enrichment = pd.read_csv(enrichment_fpath, sep='\t', index_col='Name')

# read gene sets
with open(ingmt_fpath) as f:
    D = dict()
    for line in f:
        lin = line.rstrip().split('\t')
        d = {lin[0]: lin[2:]}
        D.update(d)
gsets = pd.DataFrame(pd.Series(D), columns=['Genes'])

# join enrichment and gene sets
enrichment = pd.concat([enrichment, gsets], axis=1)
enrichment['numGenes'] = enrichment.Genes.apply(lambda l: len(l))
enrichment

,chi2Pvalue,empPvalue,Genes,numGenes
PPI-STRING_Consensus_mod272,0.000020,0.00074,"[LAIR1, SLA, FCAR, FCGR3A, ITK, CD58, C7orf43,...",67
PPI-STRING_Consensus_mod248,0.000110,0.00094,"[PIK3R1, SKAP2, SH3BP5, SHC4, PDGFB, DOK3, HCL...",74
PPI-STRING_Consensus_mod191,0.000894,0.00115,"[AVPR1B, GNG2, GNAQ, APP, SPIN3, CPE, PLCB3, P...",31
PPI-STRING_Consensus_mod142,0.000907,0.00248,"[HS3ST1, SDC1, B3GALT6, SDC3, SDC2, SLC35D2, H...",32
PPI-STRING_Consensus_mod328,0.001421,0.00302,"[MCOLN2, TMPRSS9, KATNAL2, ABI3, NKIRAS2, GPR1...",55
...,...,...,...,...
PPI-STRING_Consensus_mod358,0.989155,0.98939,"[CTIF, C1orf212, TSPAN31, SLC41A1, C8orf4, GPB...",45
PPI-STRING_Consensus_mod47,0.992947,0.99270,"[SPHK1, SPHK2, CERS3, CERK, SGPP2, CERS4, SGMS...",28
PPI-STRING_Consensus_mod66,0.998098,0.99812,"[ACACA, ACACB, SCP2D1, CPT1C, MS4A8B, HMGCL, A...",94
PPI-STRING_Consensus_mod9,0.998261,0.99844,"[OR1D2, OR5A1, OR4K15, OR51Q1, OR5AK2, OR5H15,...",66


In [4]:
enrichment_pretty = enrichment.copy()
enrichment_pretty['Genes'] = enrichment_pretty.Genes.apply(lambda l: ' '.join(l))
enrichment_pretty.to_csv('../../results/2022-08-22-dream-modules/enrichment.csv')

In [5]:
genes = pd.read_csv('../../results/2022-08-11-pascal/results.csv', index_col=0).fillna(0)
genes.head()

,chromosome,start,end,strand,gene_id,numSnps,pvalue,Status,knowledge,TWAS,TWAS2+,agora,agora2+,DESudhir,AD DE APOE3-APOE3,AD DE APOE4-APOE4,APOE3-4 DE neuron,APOE3-4 DE astrocyte,APOE3-4 DE microglia,rMTA-APOE3-4
CYB561,chr17,61509664,61523722,-,1534,97,1.000000e-12,DAVIES_LOWPRECISION_FAREBROTHER_FAIL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IGSF23,chr19,45116939,45140081,+,147710,204,1.000000e-12,DAVIES_LOWPRECISION_FAREBROTHER_FAIL,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PVR,chr19,45147097,45169428,+,5817,239,1.000000e-12,DAVIES_LOWPRECISION_FAREBROTHER_FAIL,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CLASRP,chr19,45542297,45574214,+,11129,247,1.000000e-12,DAVIES_LOWPRECISION_FAREBROTHER_FAIL,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RELB,chr19,45504706,45541456,+,5971,212,1.000000e-12,DAVIES_LOWPRECISION_FAREBROTHER_FAIL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
dream.module_memberships('PPI-STRING_Consensus_mod248', enrichment, genes)

['',
 'NCK2 PLCG2 CD2AP INPP5D',
 '',
 'PLCG2 ALK INPP5D BDNF CBL IRS1 GDNF',
 'INPP5D',
 'PIK3R5 ANGPT1 BDNF',
 'DAPP1 CD22',
 '',
 '',
 '',
 '',
 '']

In [7]:
gsets = genes.columns[genes.columns.to_list().index('knowledge'):].to_list()
gsets

['knowledge',
 'TWAS',
 'TWAS2+',
 'agora',
 'agora2+',
 'DESudhir',
 'AD DE APOE3-APOE3',
 'AD DE APOE4-APOE4',
 'APOE3-4 DE neuron',
 'APOE3-4 DE astrocyte',
 'APOE3-4 DE microglia',
 'rMTA-APOE3-4']

In [8]:
%connect_info

{
  "shell_port": 55434,
  "iopub_port": 55435,
  "stdin_port": 55436,
  "control_port": 55438,
  "hb_port": 55437,
  "ip": "127.0.0.1",
  "key": "f82e96e1-a5592aab54e9ae5cea48928b",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-5bf91ef1-9528-4a50-88bb-f98877a2b1e3.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
